In [1]:
import sys
import numpy as np

from sklearn.metrics import mean_squared_error

sys.path.append('..')
from higgs_inference.various.utils import format_number

In [2]:
result_dir = '../results/'

# TablePrinter class

In [3]:
class TablePrinter:
    
    def __init__(self, metric_fns=[], header=None, precisions=[]):
        
        # Functions for metrics
        self.metric_fns = metric_fns
        self.n_metrics = len(self.metric_fns)
        self.precisions = precisions if len(precisions) == self.n_metrics else [2] * self.n_metrics
        
        # Total table and current block
        self.table = ''
        self.block_entries = []
        self.block_brackets = []

        # Formatting options
        self.indent = '   '
        self.col_sep = ' & '
        self.end_row = r'\\'
        self.midrule = r'\midrule'
        self.end_line = '\n'
        self.emphasis_begin = r'\mathbf{'
        self.emphasis_end = r'}'
        
        # Header
        self.table = ''
        if header is not None:
            self.table += self.indent + header + self.end_row + self.end_line
    
    
    def finalise_block(self):

        # Find best performance
        block_metrics = [line[2:] for line in self.block_entries]
        block_metrics = np.array(block_metrics)
        block_best = []
        for i in range(self.n_metrics):
            try:
                block_best.append(np.nanargmin(block_metrics[:,i]))
            except ValueError:
                block_best.append(-1)

        # Format entries
        text = ''
        for i, (line, brackets) in enumerate(zip(self.block_entries, self.block_brackets)):
            
            # Labels
            text += self.indent + line[0] + self.col_sep + line[1] + self.col_sep
            
            # Metrics
            for j in range(self.n_metrics):
                if np.isfinite(line[j + 2]):
                    if brackets[j+2]:
                        text += '(' + format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j])) + ')'
                    else:
                        text += format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j]))
                if j == len(line) - 3:
                    text += self.end_row + self.end_line
                else:
                    text += self.col_sep

        # Add to document and reset for next block
        self.table += text
        self.block_entries = []
        self.block_brackets = []
    
    
    def new_block(self):
        self.finalise_block()
        self.table += self.indent + self.midrule + self.end_line
    
    
    def add(self, col1, col2, filename, folder='parameterized'):
        
        # Label columns
        line = [col1, col2]
        brackets = [False, False]
        
        # Metrics
        for fn in self.metric_fns:
            bracket = False
            try:
                value = fn(filename, folder)
            except (IOError, ValueError):
                #print('File', filename, 'in folder', folder, 'not found')
                value = np.nan
                
            if isinstance(value, (list, tuple)):
                value, bracket = value
                
            line.append(value)
            brackets.append(bracket)

        self.block_entries.append(line)
        self.block_brackets.append(brackets)
    
    
    def print(self):
        self.finalise_block()
        return self.table

# Metrics

In [4]:
def mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return mean_squared_error(log_r_truth, log_r_estimated)
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return mean_squared_error(log_r_truth[finites], log_r_estimated[finites]), True

def mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    try:
        return mean_squared_error(log_r_truth, log_r_estimated)
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return mean_squared_error(log_r_truth[finites], log_r_estimated[finites]), True

def mse_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    try:
        return mean_squared_error(expected_log_r_truth, expected_log_r_estimated)
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites]), True

def mse_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    try:
        return mean_squared_error(expected_log_r_truth, expected_log_r_estimated)
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites]), True

def mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return mean_squared_error(t_truth, t_estimated)

def mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return mean_squared_error(t_truth, t_estimated)

def var_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    return np.var(expected_log_r_truth - expected_log_r_estimated)

def var_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    return np.var(expected_log_r_truth - expected_log_r_estimated)

# Tables

In [5]:
labels = ['AFC',
          'carl (PbP, raw)',
          'carl (PbP, cal.)',
          'carl (param., raw)',
          'carl (param., cal.)',
          'carl + score (param., raw)',
          'carl + score (param., cal.)',
          'SM score regression',
          'Regression (PbP, raw)',
          'Regression (param., raw)',
          'Regr.\ + score (param., raw)']

folders = (['afc']
          + ['point_by_point'] * 2 + ['parameterized'] * 4
          + ['score_regression']
          + ['point_by_point'] + ['parameterized'] * 2)

filenames = ['afc',
             'carl',
             'carl_calibrated',
             'carl',
             'carl_calibrated',
             'combined',
             'combined_calibrated',
             'scoreregression',
             'regression',
             'regression',
             'combinedregression']

In [6]:
table = TablePrinter([mse_log_r_trained, mse_log_r_nottrained,
                      mse_expected_log_r, mse_delta_expected_log_r],
                      precisions=[3,3,2,2])

for i, (label, filename, folder) in enumerate(zip(labels[:5], filenames[:5], folders[:5])):
    
    if i > 0:
        table.new_block()
        
    if folder == 'point_by_point':
        #table.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
        table.add(label, 'PbP (2)', filename, 'point_by_point')
        #table.add('', 'PbP (3)', filename + '_deep', 'point_by_point')
        
    elif folder == 'afc':
        #table.add(label, 'PbP (1)', filename + '_shallow', 'afc')
        table.add(label, 'PbP (2)', filename, 'afc')
        #table.add('', 'PbP (3)', filename + '_deep', 'afc')
        
    elif folder == 'score_regression':
        #table.add('', r'Fixed density est. on $t$ (1)', filename + '_score_shallow', 'score_regression')
        table.add(label, r'Fixed density est. on $t$  (2)', filename + '_score', 'score_regression')
        #table.add('', r'Fixed density est. on $t$  (3)', filename + '_score_deep', 'score_regression')
        
        #table.add('', r'Dynamic density est. on $t$  (1)', filename + '_rotatedscore_shallow', 'score_regression')
        table.add('', r'Dynamic density est. on $t$  (2)', filename + '_rotatedscore', 'score_regression')
        #table.add('', r'Dynamic density est. on $t$  (3)', filename + '_rotatedscore_deep', 'score_regression')
        
        #table.add('', r'Density est. on $t \cdot \theta$  (1)', filename + '_scoretheta_shallow', 'score_regression')
        table.add('', r'Density est. on $t \cdot \theta$  (2)', filename + '_calibrated', 'score_regression')
        #table.add('', r'Density est. on $t \cdot \theta$  (3)', filename + '_scoretheta_deep', 'score_regression')
    
    else:
        #table.add(label, 'Baseline (1)', filename + '_shallow')
        table.add(label, 'Baseline (2)', filename)
        #table.add('', 'Baseline (3)', filename + '_deep')

        #table.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
        table.add('', r'Random $\boldtheta$ (2)', filename + '_random')
        #table.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

        #table.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
        table.add('', 'Physics-aware, baseline (2)', filename + '_aware')
        #table.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

        #table.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
        table.add('', 'Physics-aware, morphing basis (2)', filename + '_aware_basis')
        #table.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table.print())

   AFC & PbP (2) & \emph{0.094} & \emph{0.119} & \emph{15.43} & \emph{13.81}\\
   \midrule
   carl (PbP, raw) & PbP (2) & \emph{0.061} & \emph{0.167} & \emph{286.25} & \emph{236.39}\\
   \midrule
   carl (PbP, cal.) & PbP (2) & \emph{0.027} & \emph{0.031} & \emph{44.75} & \emph{112.43}\\
   \midrule
   carl (param., raw) & Baseline (2) & \emph{0.025} & \emph{0.037} & \emph{19.47} & \emph{23.42}\\
    & Random $\boldtheta$ (2) & 0.040 & 0.053 & 43.28 & 125.83\\
    & Physics-aware, baseline (2) & 0.265 & (0.449) & (121.38) & (80.45)\\
    & Physics-aware, morphing basis (2) &  &  &  & \\
   \midrule
   carl (param., cal.) & Baseline (2) & \emph{0.024} & \emph{0.035} & 47.46 & 116.90\\
    & Random $\boldtheta$ (2) & 0.028 & 0.039 & \emph{45.41} & \emph{114.29}\\
    & Physics-aware, baseline (2) &  &  &  & \\
    & Physics-aware, morphing basis (2) &  &  &  & \\



/Users/johannbrehmer/anaconda3/envs/higgs_inference/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


In [7]:
table = TablePrinter([mse_log_r_trained, mse_log_r_nottrained,
                      mse_expected_log_r, mse_delta_expected_log_r],
                      precisions=[3,3,2,2])

for i, (label, filename, folder) in enumerate(zip(labels[5:8], filenames[5:8], folders[5:8])):
    
    if i > 0:
        table.new_block()
        
    if folder == 'point_by_point':
        #table.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
        table.add(label, 'PbP (2)', filename, 'point_by_point')
        #table.add('', 'PbP (3)', filename + '_deep', 'point_by_point')
        
    elif folder == 'afc':
        #table.add(label, 'PbP (1)', filename + '_shallow', 'afc')
        table.add(label, 'PbP (2)', filename, 'afc')
        #table.add('', 'PbP (3)', filename + '_deep', 'afc')
        
    elif folder == 'score_regression':
        #table.add('', r'Fixed density est. on $t$ (1)', filename + '_score_shallow', 'score_regression')
        table.add(label, r'Fixed density est. on $t$  (2)', filename + '_score', 'score_regression')
        #table.add('', r'Fixed density est. on $t$  (3)', filename + '_score_deep', 'score_regression')
        
        #table.add('', r'Dynamic density est. on $t$  (1)', filename + '_rotatedscore_shallow', 'score_regression')
        table.add('', r'Dynamic density est. on $t$  (2)', filename + '_rotatedscore', 'score_regression')
        #table.add('', r'Dynamic density est. on $t$  (3)', filename + '_rotatedscore_deep', 'score_regression')
        
        #table.add('', r'Density est. on $t \cdot \theta$  (1)', filename + '_scoretheta_shallow', 'score_regression')
        table.add('', r'Density est. on $t \cdot \theta$  (2)', filename + '_calibrated', 'score_regression')
        #table.add('', r'Density est. on $t \cdot \theta$  (3)', filename + '_scoretheta_deep', 'score_regression')
    
    else:
        #table.add(label, 'Baseline (1)', filename + '_shallow')
        table.add(label, 'Baseline (2)', filename)
        #table.add('', 'Baseline (3)', filename + '_deep')

        #table.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
        table.add('', r'Random $\boldtheta$ (2)', filename + '_random')
        #table.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

        #table.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
        table.add('', 'Physics-aware, baseline (2)', filename + '_aware')
        #table.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

        #table.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
        table.add('', 'Physics-aware, morphing basis (2)', filename + '_aware_basis')
        #table.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table.print())

   carl + score (param., raw) & Baseline (2) & \emph{0.022} & \emph{0.023} & \emph{44.81} & \emph{0.90}\\
    & Random $\boldtheta$ (2) & 0.047 & 0.065 & 48.25 & 149.13\\
    & Physics-aware, baseline (2) & 0.234 & 0.291 & (61.93) & (78.01)\\
    & Physics-aware, morphing basis (2) &  &  &  & \\
   \midrule
   carl + score (param., cal.) & Baseline (2) & \emph{0.013} & \emph{0.015} & \emph{47.72} & \emph{116.17}\\
    & Random $\boldtheta$ (2) & 0.022 & 0.030 & 51.22 & 124.21\\
    & Physics-aware, baseline (2) &  &  &  & \\
    & Physics-aware, morphing basis (2) &  &  &  & \\
   \midrule
   Score regression & Fixed density est. on $t$  (2) &  &  & 12476633.36 & 25282974.38\\
    & Dynamic density est. on $t$  (2) &  &  & 30353.84 & 92327.85\\
    & Density est. on $t \cdot \theta$  (2) & \emph{0.507} & \emph{0.459} & \emph{16.17} & \emph{29.46}\\



In [8]:
table = TablePrinter([mse_log_r_trained, mse_log_r_nottrained,
                      mse_expected_log_r, mse_delta_expected_log_r],
                      precisions=[3,3,2,2])

for i, (label, filename, folder) in enumerate(zip(labels[8:], filenames[8:], folders[8:])):
    
    if i > 0:
        table.new_block()
        
    if folder == 'point_by_point':
        #table.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
        table.add(label, 'PbP (2)', filename, 'point_by_point')
        #table.add('', 'PbP (3)', filename + '_deep', 'point_by_point')
        
    elif folder == 'afc':
        #table.add(label, 'PbP (1)', filename + '_shallow', 'afc')
        table.add(label, 'PbP (2)', filename, 'afc')
        #table.add('', 'PbP (3)', filename + '_deep', 'afc')
        
    elif folder == 'score_regression':
        #table.add('', r'Fixed density est. on $t$ (1)', filename + '_score_shallow', 'score_regression')
        table.add(label, r'Fixed density est. on $t$  (2)', filename + '_score', 'score_regression')
        #table.add('', r'Fixed density est. on $t$  (3)', filename + '_score_deep', 'score_regression')
        
        #table.add('', r'Dynamic density est. on $t$  (1)', filename + '_rotatedscore_shallow', 'score_regression')
        table.add('', r'Dynamic density est. on $t$  (2)', filename + '_rotatedscore', 'score_regression')
        #table.add('', r'Dynamic density est. on $t$  (3)', filename + '_rotatedscore_deep', 'score_regression')
        
        #table.add('', r'Density est. on $t \cdot \theta$  (1)', filename + '_scoretheta_shallow', 'score_regression')
        table.add('', r'Density est. on $t \cdot \theta$  (2)', filename + '_calibrated', 'score_regression')
        #table.add('', r'Density est. on $t \cdot \theta$  (3)', filename + '_scoretheta_deep', 'score_regression')
    
    else:
        #table.add(label, 'Baseline (1)', filename + '_shallow')
        table.add(label, 'Baseline (2)', filename)
        #table.add('', 'Baseline (3)', filename + '_deep')

        #table.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
        table.add('', r'Random $\boldtheta$ (2)', filename + '_random')
        #table.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

        #table.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
        table.add('', 'Physics-aware, baseline (2)', filename + '_aware')
        #table.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

        #table.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
        table.add('', 'Physics-aware, morphing basis (2)', filename + '_aware_basis')
        #table.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table.print())

   Regression (PbP, raw) & PbP (2) & \emph{0.002} & \emph{0.003} & \emph{3.15} & \emph{3.21}\\
   \midrule
   Regression (param., raw) & Baseline (2) & \emph{0.004} & \emph{0.005} & \emph{1.11} & \emph{0.95}\\
    & Random $\boldtheta$ (2) & 0.005 & 0.008 & 2.50 & 3.01\\
    & Physics-aware, baseline (2) & 0.027 & 0.378 & 183.07 & 174.79\\
    & Physics-aware, morphing basis (2) & 0.015 & 85.603 & 9484.81 & 17392.32\\
   \midrule
   Regr.\ + score (param., raw) & Baseline (2) & \emph{0.005} & \emph{0.006} & \emph{1.69} & 1.30\\
    & Random $\boldtheta$ (2) & 0.005 & 0.006 & 2.42 & \emph{1.02}\\
    & Physics-aware, baseline (2) & 0.054 & 0.335 & 178.66 & 180.78\\
    & Physics-aware, morphing basis (2) &  &  &  & \\



In [11]:
labels = ['carl',
          'Score',
          'carl + score',
          'Regression',
          'Regr.\ + score']

filenames = ['carl',
             'score',
             'combined',
             'regression',
             'combinedregression']

table4 = TablePrinter([mse_score_trained, mse_score_nottrained],
                      precisions=[2,2])

for i, (label, filename) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        table4.new_block()
        
    #table4.add(label, 'Baseline (1)', filename + '_shallow')
    table4.add(label, 'Baseline (2)', filename)
    #table4.add('', 'Baseline (3)', filename + '_deep')
    
    #table4.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
    table4.add('', r'Random $\boldtheta$ (2)', filename + '_random')
    #table4.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

    #table4.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
    table4.add('', 'Physics-aware, baseline (2)', filename + '_aware')
    #table4.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

    #table4.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
    table4.add('', 'Physics-aware, morphing basis (2)', filename + '_aware_basis')
    #table4.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table4.print())

   carl & Baseline (2) & \emph{0.67} & \emph{0.95}\\
    & Random $\boldtheta$ (2) & 0.86 & 1.33\\
    & Physics-aware, baseline (2) & 221.12 & 1913.99\\
    & Physics-aware, morphing basis (2) &  & \\
   \midrule
   Score & Baseline (2) & \emph{0.07} & \emph{0.26}\\
    & Random $\boldtheta$ (2) & 0.38 & 0.73\\
    & Physics-aware, baseline (2) & 1.06 & 0.94\\
    & Physics-aware, morphing basis (2) &  & \\
   \midrule
   carl + score & Baseline (2) & \emph{0.10} & \emph{0.25}\\
    & Random $\boldtheta$ (2) & 0.36 & 0.72\\
    & Physics-aware, baseline (2) & 0.43 & 0.94\\
    & Physics-aware, morphing basis (2) &  & \\
   \midrule
   Regression & Baseline (2) & \emph{0.13} & \emph{0.26}\\
    & Random $\boldtheta$ (2) & 0.17 & 0.34\\
    & Physics-aware, baseline (2) & 6.85 & 650.47\\
    & Physics-aware, morphing basis (2) & 782.61 & 8.87\\
   \midrule
   Regr.\ + score & Baseline (2) & \emph{0.10} & \emph{0.20}\\
    & Random $\boldtheta$ (2) & 0.14 & 0.31\\
    & Physics-aware, ba